In [1]:
#import necessary libraries
import numpy as np
import math
import pandas
from pandas import read_csv
from pandas import concat
import random
from decimal import *
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.metrics import f1_score,confusion_matrix,roc_auc_score


import warnings

#suppress warnings
warnings.filterwarnings('ignore')



In [2]:
#activation function and partial derivatives

#define sigmoid activation function
def sigmoid(x):
	return 1 / (1.0 + np.exp(-x))

# neuron derivatives
def n_derivatives(z):
	return z * (1.0 - z)


#evaluation functions
def evaluation(y_test,y_pred):

	acc = accuracy_score(y_test,y_pred)
	rcl = recall_score(y_test,y_pred)
	f1 = f1_score(y_test,y_pred)
	auc_score = roc_auc_score(y_test,y_pred)
	prec_score = precision_score(y_test,y_pred)

	metric_dict={'accuracy': round(acc,3),
	           'recall': round(rcl,3),
	           'F1 score': round(f1,3),
	           'auc score': round(auc_score,3),
	           'precision': round(prec_score,3) 
	          }

	return print(metric_dict)

#normalize the prediction 
def normalize(pred):
	ret = np.array((pred))

	for i in range(len(pred)):
		if pred[i]>=0.5: #tp
			ret[i]=1
		else:
			ret[i]=0

	return ret


In [3]:
#Neural Network Class

class NeuralNetwork(object):
	"""Make a 3-layer network (5-4-1)"""
	def __init__(self, X, Y):
		#initialized important variables

		self.inputs = X

		self.hidden_weights = np.random.uniform(low=-0.1, high=0.1, size=(self.inputs.shape[1],4))#weights of hidden layer
		self.hidden_bias = np.random.uniform(low=-0.1, high=0.1, size=(4,)) #bias of hidden layer

		self.output_weights = np.random.uniform(low=-0.1, high=0.1, size=(4,1)) #weights of output layer
		self.ouput_bias = np.random.uniform(low=-0.1, high=0.1, size=(1,))  #bias of the output layer

		#------------------------------------------------------------------------------------------------------------#
		# self.hidden_weights = np.full((self.inputs.shape[1],3), 0.5, dtype=float) #weights of hidden layer
		# self.hidden_bias = np.full((3,), 0.5, dtype=float) #bias of hidden layer

		# self.output_weights = np.full((3,1), 0.5, dtype=float) #weights of output layer
		# self.ouput_bias = np.full((1,), 0.5, dtype=float) #bias of the output layer
		#------------------------------------------------------------------------------------------------------------#


		self.a_out = Y.reshape(len(Y),1)#actual outputs
		self.p_outputs = np.zeros(Y.shape) #holds predicted outputs

		print("Hidden Nodes Weights: \n",self.hidden_weights, "\nHidden Nodes Biases: \n",self.hidden_bias)
        
		print("Output Nodes Weights: \n",self.output_weights, "\nOutput Nodes Biases: \n",self.ouput_bias)

	def predict(self,X):
		#predict using a different input
		self.inputs = X
		return normalize(self.forward_pass())

	def forward_pass(self):
		self.hidden_nodes = sigmoid(np.add(np.dot(self.inputs, self.hidden_weights),self.hidden_bias)) #contains hidden node inputs
		outputs = sigmoid(np.add(np.dot(self.hidden_nodes, self.output_weights),self.ouput_bias)) #contain output prediction
		return outputs

	def back_propagration(self,l_rate):
		#output error and delta
		self.output_error = self.a_out - self.p_outputs
		self.output_delta = self.output_error * n_derivatives(self.p_outputs)

		#hidden error and delta
		self.hidden_error = np.dot(self.output_delta,self.output_weights.T)
		self.hidden_delta = self.hidden_error*n_derivatives(self.hidden_nodes)

		#update weights
		self.output_weights += np.dot(self.hidden_nodes.T,self.output_delta) * l_rate
		self.hidden_weights += np.dot(self.inputs.T,self.hidden_delta) * l_rate

		# #update biases
		self.hidden_bias  += np.sum(self.hidden_delta,axis=0) * l_rate
		self.ouput_bias += np.sum(self.output_delta,axis=0)  *l_rate
		return

	def train(self):
		self.p_outputs = self.forward_pass()
		self.back_propagration(0.00001) #use learning rate
		return

In [4]:
#Prepare the Datasets

#CLEAN THE DATASET
# load data
test1 = read_csv('datatest.txt', header=0, index_col=0, parse_dates=True, squeeze=True)
train = read_csv('datatraining.txt', header=0, index_col=0, parse_dates=True, squeeze=True)
test2 = read_csv('datatest2.txt', header=0, index_col=0, parse_dates=True, squeeze=True)

values = train.values
#separate inputs and output values
INPUT, OUTPUT = values[:, 1:-1], values[:, -1]
X, Y = np.array((INPUT),dtype=float), np.array((OUTPUT),dtype=float)

values1 = test1.values
#separate inputs and output values
INPUT1, OUTPUT1 = values1[:, 1:-1], values1[:, -1]
X1, Y1 = np.array((INPUT1),dtype=float), np.array((OUTPUT1),dtype=float)

values2 = test2.values
#separate inputs and output values
INPUT2, OUTPUT2 = values2[:, 1:-1], values2[:, -1]
X2, Y2 = np.array((INPUT2),dtype=float), np.array((OUTPUT2),dtype=float)

print(train.info())


"""
Independent Variables:
-Temperature, Humidity, Light, CO2, HumidityRatio
Dependent Variable: 
-Occupancy
"""

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8143 entries, 1 to 8143
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           8143 non-null   object 
 1   Temperature    8143 non-null   float64
 2   Humidity       8143 non-null   float64
 3   Light          8143 non-null   float64
 4   CO2            8143 non-null   float64
 5   HumidityRatio  8143 non-null   float64
 6   Occupancy      8143 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 508.9+ KB
None


'\nIndependent Variables:\n-Temperature, Humidity, Light, CO2, HumidityRatio\nDependent Variable: \n-Occupancy\n'

In [5]:
#Create the Neural Network and Train using the training dataset

#epoch = 1500 with learning rate of 0.00001

NN = NeuralNetwork(X, Y)

for epoch in range(1500):	
    if epoch % 100==0: 
        print ("Epoch " + str(epoch) + " Loss: " + str(np.mean(np.square(NN.a_out- NN.p_outputs)))) # mean squared error for loss
    NN.train()


Hidden Nodes Weights: 
 [[-0.00606723 -0.0909193  -0.07372386  0.09959427]
 [ 0.05901565  0.03703342 -0.07642853  0.01572788]
 [ 0.09361881 -0.02215084 -0.09135864  0.08583708]
 [ 0.00293935  0.0224781  -0.07544568 -0.02731063]
 [-0.0432646  -0.05462081  0.04466519 -0.07196674]] 
Hidden Nodes Biases: 
 [-0.06196543  0.06961126 -0.08614653  0.01436121]
Output Nodes Weights: 
 [[ 0.04734569]
 [ 0.06602199]
 [-0.03927068]
 [ 0.08025851]] 
Output Nodes Biases: 
 [0.02000303]
Epoch 0 Loss: 0.2123296082524868
Epoch 100 Loss: 0.16059377750194095
Epoch 200 Loss: 0.13100918819636348
Epoch 300 Loss: 0.11307521671620871
Epoch 400 Loss: 0.10249366489903393
Epoch 500 Loss: 0.09204661413405116
Epoch 600 Loss: 0.08340519123104284
Epoch 700 Loss: 0.07808165415531416
Epoch 800 Loss: 0.0754877168141456
Epoch 900 Loss: 0.07164916031977182
Epoch 1000 Loss: 0.0670813306376413
Epoch 1100 Loss: 0.0671409787821466
Epoch 1200 Loss: 0.06480587586101119
Epoch 1300 Loss: 0.062349668313736756
Epoch 1400 Loss: 0.06

In [6]:
#evaluate the Neural Network
print("EVALUATION TRAIN")
evaluation(NN.a_out,normalize(NN.p_outputs))
print("EVALUATION TEST1")
evaluation(Y1,NN.predict(X1))
print("EVALUATION TEST2")
evaluation(Y2,NN.predict(X2))


EVALUATION TRAIN
{'accuracy': 0.934, 'recall': 0.995, 'F1 score': 0.865, 'auc score': 0.957, 'precision': 0.765}
EVALUATION TEST1
{'accuracy': 0.979, 'recall': 1.0, 'F1 score': 0.972, 'auc score': 0.983, 'precision': 0.946}
EVALUATION TEST2
{'accuracy': 0.946, 'recall': 0.999, 'F1 score': 0.885, 'auc score': 0.965, 'precision': 0.795}


In [7]:
#sample prediction

#23.7,26.272,585.2,749.2,0.00476416302416414,1
#x = ["Temperature","Humidity","Light","CO2","HumidityRatio"]
#y = ["Actual Occupancy"]

x= [23.7,26.272,585.2,749.2,0.00476416302416414]
y= [1]


print(normalize(NN.predict(x)),y)

[1.] [1]
